In [ ]:
import requests
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
API_URL = os.getenv("API_URL")

python-dotenv could not parse statement starting at line 2


In [ ]:
API_URL

In [5]:
# put your token here
headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

response = query({
    "messages": [
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
    "model": "openai/gpt-oss-20b:nebius"
})

print(response["choices"][0]["message"])

MissingSchema: Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?

In [ ]:

# 2. SETUP THE COUNCIL MEMBERS (MODELS)
# -------------------------------------
# Note: "GPT-OSS" and "Kimi" are not standard hosted IDs on the public router.
# I have substituted them with valid, high-performance open models.
# You can swap these strings for any model ID on the HF Hub.
council_members = [
    "meta-llama/Meta-Llama-3.1-8B-Instruct",  # The balanced leader
    "Qwen/Qwen2.5-72B-Instruct",              # The logic/math expert (Qwen 2.5 is current SOTA)
    "THUDM/glm-4-9b-chat",                    # The GLM representative
    "mistralai/Mistral-7B-Instruct-v0.3",     # A solid, concise member
    "google/gemma-2-9b-it"                    # Google's open weight contributor
]

# 3. DEFINE THE INFERENCE FUNCTIONS
# ---------------------------------
import requests
import json

headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

def ask_council(prompt, model_id):
    """
    Sends the user prompt to a specific model via HF Inference API.
    """
    payload = {
        "model": model_id,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 500, # Limit response size for quick testing
        "stream": False
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status() # Check for HTTP errors
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# 4. RUN THE COUNCIL
# ------------------
user_question = "What is the capital of France, and give me one fun fact about it?"

print(f"--- 🏛️ THE COUNCIL IS CONVENING 🏛️ ---")
print(f"QUESTION: {user_question}\n")

for member in council_members:
    print(f"👉 Council Member speaking: {member}")
    
    # Query the model
    result = ask_council(user_question, member)
    
    # Extract answer safely
    if "choices" in result:
        answer = result["choices"][0]["message"]["content"]
        print(f"💬 ANSWER:\n{answer}\n")
    else:
        # Sometimes models are loading (cold start), this handles that.
        print(f"⚠️ ERROR/LOADING: {result}\n")
        
    print("-" * 50)